In [1]:
# ================================================
# ✅ 1️⃣ LIBRARIES & SETUP
# ================================================
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# ================================================
# ✅ 2️⃣ PATHS
# ================================================
image_dir = "/kaggle/input/basem/images"
input_csv = "/kaggle/input/basem/dataset.csv"

# ================================================
# ✅ 3️⃣ LOAD & PREPROCESS CSV
# ================================================
df = pd.read_csv(input_csv)

existing_data = []
for _, row in df.iterrows():
    image_filename = row['image_path']
    full_image_path = os.path.join(image_dir, image_filename)
    if os.path.exists(full_image_path):
        label_converted = row['label 2'] - 1
        existing_data.append({
            'Image_path': full_image_path,
            'Label_Sentiment': label_converted
        })

processed_df = pd.DataFrame(existing_data)

# ================================================
# ✅ 4️⃣ DATA SPLITTING
# ================================================
train_df, temp_df = train_test_split(processed_df, test_size=0.3, stratify=processed_df['Label_Sentiment'], random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=1/3, stratify=temp_df['Label_Sentiment'], random_state=42)

for df_name, df_ in [('train', train_df), ('test', test_df), ('val', val_df)]:
    df_['label'] = df_['Label_Sentiment']
    df_.to_csv(f'/kaggle/working/{df_name}_vision_only.csv', index=False)

# ================================================
# ✅ 5️⃣ IMAGE TRANSFORMS
# ================================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ================================================
# ✅ 6️⃣ DATASET CLASS
# ================================================
class VisionOnlyDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['Image_path']).convert('RGB')
        label = row['label']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# ================================================
# ✅ 7️⃣ DATALOADERS
# ================================================
batch_size = 16

train_loader = DataLoader(
    VisionOnlyDataset(train_df, train_transform), 
    batch_size=batch_size, 
    shuffle=True
)
val_loader = DataLoader(
    VisionOnlyDataset(val_df, val_test_transform), 
    batch_size=batch_size, 
    shuffle=False
)
test_loader = DataLoader(
    VisionOnlyDataset(test_df, val_test_transform), 
    batch_size=batch_size, 
    shuffle=False
)

# ================================================
# ✅ 8️⃣ VGG16 MODEL
# ================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class VGG16Classifier(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG16Classifier, self).__init__()
        # Load pre-trained VGG16
        self.vgg16 = models.vgg16(pretrained=True)
        
        # Freeze early layers (optional - comment out if you want to fine-tune all layers)
        for param in self.vgg16.features.parameters():
            param.requires_grad = False
        
        # Replace the classifier
        self.vgg16.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        return self.vgg16(x)

model = VGG16Classifier(num_classes=3).to(device)

# ================================================
# ✅ 9️⃣ LOSS & OPTIMIZER
# ================================================
class_weights = train_df['label'].value_counts().sort_index().tolist()
total = sum(class_weights)
weights = [total / c for c in class_weights]
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(weights).to(device))
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

# ================================================
# ✅ 🔟 TRAINING LOOP
# ================================================
num_epochs = 20
patience = 3
patience_counter = 0
best_val_loss = float('inf')

print("Starting training...")
for epoch in range(num_epochs):
    # ============================================================
    # TRAINING PHASE
    # ============================================================
    model.train()
    total_train_loss = 0
    train_predictions = []
    train_labels = []

    for images, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        
        # Store predictions for metrics
        predictions = torch.argmax(outputs, dim=1)
        train_predictions.extend(predictions.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = accuracy_score(train_labels, train_predictions)

    # ============================================================
    # VALIDATION PHASE
    # ============================================================
    model.eval()
    total_val_loss = 0
    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_val_loss += loss.item()
            
            # Store predictions for metrics
            predictions = torch.argmax(outputs, dim=1)
            val_predictions.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_labels, val_predictions)
    
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")

    # ============================================================
    # EARLY STOPPING CHECK
    # ============================================================
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_vgg16_model.pt")
        print("✅ Validation loss improved — model saved.")
    else:
        patience_counter += 1
        print(f"⏰ No improvement — patience {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break
    print("-" * 50)

# ================================================
# ✅ 1️⃣1️⃣ FINAL TEST EVALUATION
# ================================================
print("\n🔍 Loading best model for final evaluation...")
model.load_state_dict(torch.load("best_vgg16_model.pt"))
model.eval()

test_predictions = []
test_labels = []
total_test_loss = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Final Test Evaluation"):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        total_test_loss += loss.item()
        
        predictions = torch.argmax(outputs, dim=1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Calculate final metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, test_predictions, average='weighted')
cm = confusion_matrix(test_labels, test_predictions)

print("\n📊 FINAL TEST RESULTS (Vision-Only VGG16):")
print("=" * 50)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test Loss: {total_test_loss/len(test_loader):.4f}")
print(f"\nConfusion Matrix:\n{cm}")

# ================================================
# ✅ 1️⃣2️⃣ DETAILED METRICS BY CLASS
# ================================================
precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
    test_labels, test_predictions, average=None
)

print("\n📈 DETAILED METRICS BY CLASS:")
print("=" * 50)
for i in range(len(precision_per_class)):
    print(f"Class {i}: Precision={precision_per_class[i]:.4f}, Recall={recall_per_class[i]:.4f}, F1={f1_per_class[i]:.4f}, Support={support[i]}")

Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 232MB/s]  


Starting training...


Validation Epoch 1: 100%|██████████| 29/29 [00:13<00:00,  2.12it/s]


Epoch [1/20]
Train Loss: 1.0923 | Train Acc: 0.4895
Val Loss: 0.9162 | Val Acc: 0.5698
✅ Validation loss improved — model saved.
--------------------------------------------------


...

Epoch [4/20]
Train Loss: 0.7809 | Train Acc: 0.6502
Val Loss: 1.0010 | Val Acc: 0.6142
⏰ No improvement — patience 2/3
--------------------------------------------------


Validation Epoch 5: 100%|██████████| 29/29 [00:11<00:00,  2.56it/s]


Epoch [5/20]
Train Loss: 0.7105 | Train Acc: 0.6746
Val Loss: 0.9971 | Val Acc: 0.6120
⏰ No improvement — patience 3/3
🛑 Early stopping triggered at epoch 5

🔍 Loading best model for final evaluation...


Final Test Evaluation: 100%|██████████| 57/57 [00:26<00:00,  2.14it/s]


📊 FINAL TEST RESULTS (Vision-Only VGG16):
Test Accuracy: 0.5876
Test Precision: 0.6131
Test Recall: 0.5876
Test F1-Score: 0.5920
Test Loss: 0.8738

Confusion Matrix:
[[221 136  45]
 [ 62 228  63]
 [ 19  47  81]]

📈 DETAILED METRICS BY CLASS:
Class 0: Precision=0.7318, Recall=0.5498, F1=0.6278, Support=402
Class 1: Precision=0.5547, Recall=0.6459, F1=0.5969, Support=353
Class 2: Precision=0.4286, Recall=0.5510, F1=0.4821, Support=147
